In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
import numpy as np
import cv2
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mtcnn import MTCNN
import mediapipe as mp
import matplotlib.pyplot as plt

class StopOnValLoss(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        val_loss = logs.get('val_loss')
        if val_loss is not None and val_loss < 0.7:
            print(f"\nEpoch {epoch + 1}: Validation loss below 0.7 ({val_loss:.4f}), stopping training.")
            self.model.stop_training = True

# Initialize face detection
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Initialize MTCNN for registration
mtcnn_detector = MTCNN()

# Selected important facial landmarks (indices from MediaPipe's 468 landmarks)
IMPORTANT_LANDMARKS = [
    10,   # Forehead
    152,  # Chin
    234,  # Left cheek
    454,  # Right cheek
    168,  # Nose tip
    33,   # Left eyebrow
    263,  # Right eyebrow
    61,   # Left eye
    291,  # Right eye
    0,    # Nose bridge
    17,   # Upper lip
    57,   # Mouth left corner
    287,  # Mouth right corner
]

def check_nim_exists(nim, dataset_dir):
    """Check if NIM folder already exists"""
    nim_folder = os.path.join(dataset_dir, nim)
    return os.path.exists(nim_folder)

def extract_mediapipe_landmarks(image):
    """Extract and customize facial landmarks with distance features"""
    results = face_mesh.process(image)
    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        
        # Get selected landmarks
        selected_landmarks = []
        for idx in IMPORTANT_LANDMARKS:
            landmark = face_landmarks.landmark[idx]
            selected_landmarks.append([landmark.x, landmark.y, landmark.z])
        
        # Calculate center point
        center_x = sum(lm[0] for lm in selected_landmarks)/len(selected_landmarks)
        center_y = sum(lm[1] for lm in selected_landmarks)/len(selected_landmarks)
        center_z = sum(lm[2] for lm in selected_landmarks)/len(selected_landmarks)
        
        # Enhanced features
        features = []
        
        # 1. Normalized coordinates relative to center
        for lm in selected_landmarks:
            features.extend([
                lm[0] - center_x,  # X coordinate relative to center
                lm[1] - center_y,  # Y coordinate relative to center
                lm[2] - center_z   # Z coordinate relative to center
            ])
        
        # 2. Distance between key facial points
        def calc_distance(idx1, idx2):
            lm1 = selected_landmarks[idx1]
            lm2 = selected_landmarks[idx2]
            return ((lm1[0]-lm2[0])**2 + (lm1[1]-lm2[1])**2 + (lm1[2]-lm2[2])**2)**0.5
        
        # Add important distances
        features.extend([
            calc_distance(0, 1),   # Forehead to chin
            calc_distance(3, 4),   # Left cheek to right cheek
            calc_distance(5, 6),   # Left eyebrow to right eyebrow
            calc_distance(7, 8),   # Left eye to right eye
            calc_distance(9, 10),  # Nose bridge to nose tip
            calc_distance(11, 12)  # Mouth left to right corner
        ])
        
        # 3. Facial ratios (important for distinguishing faces)
        features.extend([
            calc_distance(0, 1) / calc_distance(3, 4),  # Face height/width ratio
            calc_distance(5, 6) / calc_distance(7, 8),  # Eyebrow/eye width ratio
            calc_distance(9, 10) / calc_distance(0, 1)  # Nose length/face height
        ])
        
        # 4. Angle features
        def calc_angle(idx1, idx2, idx3):
            a = np.array(selected_landmarks[idx1])
            b = np.array(selected_landmarks[idx2])
            c = np.array(selected_landmarks[idx3])
            
            ba = a - b
            bc = c - b
            
            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
            return np.arccos(cosine_angle)
        
        features.extend([
            calc_angle(0, 9, 1),   # Forehead-nose-chin angle
            calc_angle(3, 4, 2),   # Cheek-nose-cheek angle
            calc_angle(5, 7, 6)    # Eyebrow-eye-eyebrow angle
        ])
        
        return np.array(features)
    return None

def load_dataset(dataset_dir):
    """Load dataset with enhanced landmark features from all users"""
    X, y = [], []
    print(f"\nLoading dataset from {dataset_dir}")
    
    # Get all NIM folders
    nim_folders = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f))]
    
    if not nim_folders:
        print("No registered users found in dataset!")
        return np.array(X), np.array(y)
    
    for nim in nim_folders:
        nim_path = os.path.join(dataset_dir, nim)
        image_path = os.path.join(nim_path, "image")
        
        if not os.path.exists(image_path):
            continue
            
        print(f"Processing NIM: {nim}")
        image_files = [f for f in os.listdir(image_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for img_name in image_files:
            img_path = os.path.join(image_path, img_name)
            image = cv2.imread(img_path)
            
            if image is None:
                continue
                
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            landmarks = extract_mediapipe_landmarks(image_rgb)
            
            if landmarks is not None:
                X.append(landmarks)
                y.append(nim)
    
    print(f"\nDataset Summary:")
    print(f"Total samples loaded: {len(X)}")
    print(f"Total unique NIMs: {len(set(y))}")
    
    return np.array(X), np.array(y)

def register_new_user(dataset_dir):
    """Register new user with automatic capture for all angles (5 images each) with user prompt between angles"""
    while True:
        nim = input("\nEnter NIM to register (or 'q' to quit): ").strip()
        if nim.lower() == 'q':
            return False, None, None
            
        if not nim.isdigit():
            print("NIM must contain only numbers!")
            continue
            
        if check_nim_exists(nim, dataset_dir):
            print(f"\nNIM {nim} already exists!")
            print("1. Register different NIM")
            print("2. Add more images to existing NIM")
            choice = input("Choose option (1/2): ")
            
            if choice == '1':
                continue
            elif choice == '2':
                print(f"\nAdding more images to NIM {nim}")
            else:
                print("Invalid choice!")
                continue
        else:
            print(f"\nRegistering new NIM: {nim}")
            
        # Create folder structure
        nim_path = os.path.join(dataset_dir, nim)
        image_path = os.path.join(nim_path, "image")
        os.makedirs(image_path, exist_ok=True)
        
        # Capture images for different angles (5 images each)
        angles = [
            ("front", "Please face the camera directly (5 images will be captured automatically) - Press SPACE when ready"),
            ("up", "Please look upwards (5 images will be captured automatically) - Press SPACE when ready"),
            ("down", "Please look downwards (5 images will be captured automatically) - Press SPACE when ready"),
            ("left", "Please turn your head to the LEFT (5 images will be captured automatically) - Press SPACE when ready"),
            ("right", "Please turn your head to the RIGHT (5 images will be captured automatically) - Press SPACE when ready")
        ]
        
        cap = cv2.VideoCapture(0)
        captured_images = 0
        
        for angle_name, instruction in angles:
            print(f"\n{instruction}")
            
            # Wait for user to be ready (SPACE key)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Display instruction
                cv2.putText(frame, instruction, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, "Press SPACE to start capture, ESC to skip", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                cv2.imshow('Registration', frame)
                
                key = cv2.waitKey(1)
                if key == 27:  # ESC to skip this angle
                    print(f"Skipping {angle_name} angle")
                    break
                elif key == 32:  # SPACE to start capturing
                    print(f"Capturing 5 {angle_name} images automatically...")
                    
                    # Automatic capture for this angle (5 images)
                    for i in range(5):
                        time.sleep(0.5)  # Reduced wait time between captures
                        
                        ret, frame = cap.read()
                        if not ret:
                            break
                            
                        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        results = mtcnn_detector.detect_faces(frame_rgb)
                        
                        if len(results) > 0:
                            x, y, w, h = results[0]['box']
                            # Expand bounding box
                            x = max(0, x - int(w * 0.2))
                            y = max(0, y - int(h * 0.2))
                            w = min(frame.shape[1] - x, int(w * 1.4))
                            h = min(frame.shape[0] - y, int(h * 1.4))
                            
                            face = frame_rgb[y:y+h, x:x+w]
                            face_resized = cv2.resize(face, (160, 160))
                            
                            # Save image with timestamp to ensure unique filenames
                            timestamp = int(time.time() * 1000)
                            img_path = os.path.join(image_path, f"{nim}_{angle_name}_{timestamp}_{i}.jpg")
                            cv2.imwrite(img_path, cv2.cvtColor(face_resized, cv2.COLOR_RGB2BGR))
                            captured_images += 1
                            
                            # Show countdown and feedback
                            cv2.putText(frame, f"Captured {i+1}/5", (x, y-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            cv2.imshow('Registration', frame)
                            cv2.waitKey(300)  # Reduced display time
                        else:
                            print(f"No face detected for {angle_name} image {i+1}")
                            continue  # Skip this capture but continue trying
                    
                    print(f"Finished capturing {angle_name} angle")
                    break
        
        cap.release()
        cv2.destroyAllWindows()
        
        print(f"\nRegistration completed for NIM: {nim}")
        print(f"Captured {captured_images} images in total")
        
        # Verify we captured exactly 25 images (5 angles × 5 images)
        if captured_images < 25:
            print(f"Warning: Only captured {captured_images} images (expected 25)")
        
        # Train model after registration
        print("\nStarting automatic model training with updated dataset...")
        model, label_encoder = train_model(dataset_dir)
        
        return True, model, label_encoder


def build_model(input_shape, num_classes):
    """Build the recognition model with enhanced features"""
    model = Sequential([
        Dense(512, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=l2(0.005)),
        BatchNormalization(),
        Dropout(0.3),



No existing model found or error loading model

==== Face Recognition System ====
1. Register new user
2. Train/retrain model with all data
3. Real-time recognition (all users)
4. Exit
Invalid choice!

==== Face Recognition System ====
1. Register new user
2. Train/retrain model with all data
3. Real-time recognition (all users)
4. Exit
Invalid choice!

==== Face Recognition System ====
1. Register new user
2. Train/retrain model with all data
3. Real-time recognition (all users)
4. Exit
Invalid choice!

==== Face Recognition System ====
1. Register new user
2. Train/retrain model with all data
3. Real-time recognition (all users)
4. Exit
Invalid choice!

==== Face Recognition System ====
1. Register new user
2. Train/retrain model with all data
3. Real-time recognition (all users)
4. Exit
Invalid choice!

==== Face Recognition System ====
1. Register new user
2. Train/retrain model with all data
3. Real-time recognition (all users)
4. Exit
Invalid choice!

==== Face Recognition Syste

In [ ]:
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

def train_model(dataset_dir):
    """Train the model with all available data"""
    X, y = load_dataset(dataset_dir)
    
    if len(X) == 0:
        print("\nNo training data available!")
        return None, None
        
    # Encode labels (NIMs)
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    
    # Split dataset (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)
    
    # Calculate class weights to handle imbalanced data
    class_weights = class_weight.compute_class_weight(
        'balanced', classes=np.unique(y_encoded), y=y_encoded)
    class_weights = dict(enumerate(class_weights))
    
    # Build model
    model = build_model((X.shape[1],), len(label_encoder.classes_))
    
    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10),
        ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True),
        StopOnValLoss()
    ]
    
    # Train model
    print("\nTraining model with all available data...")
    history = model.fit(
        X_train, y_train,
        epochs=500,
        batch_size=32,
        validation_data=(X_val, y_val),
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluate
    loss, acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"\nTraining completed. Validation accuracy: {acc*100:.2f}%")
    
    # Save label encoder
    np.save('label_encoder.npy', label_encoder.classes_)
    
    return model, label_encoder

def realtime_recognition(model, label_encoder):
    """Real-time recognition for all registered users"""
    if model is None or label_encoder is None:
        print("Model not trained yet! Please train first.")
        return
        
    cap = cv2.VideoCapture(0)
    label_map = {i:name for i,name in enumerate(label_encoder.classes_)}
    
    # Load face detection model
    face_detector = mp.solutions.face_detection.FaceDetection(
        model_selection=1, min_detection_confidence=0.5)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = face_detector.process(frame_rgb)
        
        if results.detections:
            for detection in results.detections:
                # Get bounding box
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                             int(bboxC.width * iw), int(bboxC.height * ih)
                
                # Extract face ROI
                face_roi = frame_rgb[y:y+h, x:x+w]
                
                # Extract landmarks and predict
                landmarks = extract_mediapipe_landmarks(face_roi)
                
                if landmarks is not None:
                    features = landmarks.reshape(1, -1)
                    predictions = model.predict(features, verbose=0)
                    idx = np.argmax(predictions)
                    confidence = np.max(predictions)
                    
                    # Get NIM label
                    nim = label_map[idx]
                    
                    # Draw results
                    color = (0, 255, 0) if confidence > 0.7 else (0, 255, 255) if confidence > 0.5 else (0, 0, 255)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                    cv2.putText(frame, f"NIM: {nim} ({confidence*100:.1f}%)", 
                               (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        cv2.imshow('Face Recognition - All Registered Users', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

def extract_face_and_landmarks(image_rgb):
    """Extract face and enhanced landmarks for real-time detection"""
    results = face_mesh.process(image_rgb)
    if results.multi_face_landmarks:
        # Get face bounding box
        landmarks = results.multi_face_landmarks[0].landmark
        xs = [lm.x for lm in landmarks]
        ys = [lm.y for lm in landmarks]
        
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)
        
        h, w, _ = image_rgb.shape
        x1, y1 = int(x_min * w), int(y_min * h)
        x2, y2 = int(x_max * w), int(y_max * h)
        
        # Extract enhanced landmarks
        landmark_features = extract_mediapipe_landmarks(image_rgb)
        
        return landmark_features, (x1, y1, x2-x1, y2-y1)
    
    return None, None

def main():
    dataset_dir = 'dataset'  # Folder structure: /dataset/NIM/image/
    os.makedirs(dataset_dir, exist_ok=True)
    
    # Initialize model and label encoder
    model = None
    label_encoder = None
    
    # Try to load existing model and label encoder
    try:
        model = load_model('best_model.keras')
        label_encoder = LabelEncoder()
        label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)
        print("\nLoaded existing trained model with", len(label_encoder.classes_), "registered users")
    except:
        print("\nNo existing model found or error loading model")
        
        # If dataset exists but no model, train new model
        if len(os.listdir(dataset_dir)) > 0:
            print("Found existing dataset, training new model...")
            model, label_encoder = train_model(dataset_dir)
    
    while True:
        print("\n==== Face Recognition System ====")
        print("1. Register new user")
        print("2. Train/retrain model with all data")
        print("3. Real-time recognition (all users)")
        print("4. Exit")
        
        choice = input("Select option: ")
        
        if choice == '1':
            success, new_model, new_label_encoder = register_new_user(dataset_dir)
            if success:
                model = new_model
                label_encoder = new_label_encoder
        elif choice == '2':
            model, label_encoder = train_model(dataset_dir)
        elif choice == '3':
            realtime_recognition(model, label_encoder)
        elif choice == '4':
            print("Exiting program...")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()